In [2]:

!pip install braintrust autoevals openai

In [30]:
import os
from pathlib import Path
import pandas as pd
csv_path = "../scripts/csv/default_QA.csv"


df = pd.read_csv(csv_path)
print(df.head()) 



      Unnamed: 0 Unnamed: 1            Unnamed: 2                  Unnamed: 3  \
0            提問人         類別   問題 (keywords請幫我標紅字)  Mapping到的tag (請參考tag列表第C列)   
1  LI HSIN HSIEH         其他    最近NVIDIA好像很紅，他是什麼?                      NVIDIA   
2         Ashley         其他  你可以推薦幾個適合上班族放鬆聽的節目嗎？                         NaN   
3         Ashley         其他         有沒有每日／每週推薦清單？                         NaN   
4         Ashley         其他  有適合睡前聽的放鬆 podcast 嗎？                         NaN   

                                          Unnamed: 4         Unnamed: 5  \
0                                                 答案      修改方向(上次的修改建議)   
1  你問到超熱門的話題！最近 NVIDIA 真的很紅，股價也一直在漲，很多人都在關注他們的動向！...                NaN   
2  當然可以！這邊有幾個適合下班後放鬆心情的 Podcast 節目，內容輕鬆有趣，可以讓你暫時遠...             簡單/入手/   
3  有的！我們會不定期整理一些主題清單，以下是最新一期的週推薦：\n\n📅 本週主題：《時間管理...  建立另外一個基於網頁功能的問題列表   
4  當然有，這邊是幾個睡前超推薦的 podcast，聲音溫柔、節奏舒緩，非常適合放鬆心情入眠：\...                 放鬆   

   Unnamed: 6 Unnamed: 7  
0         NaN        NaN  
1       

In [31]:
import os 
from dotenv import load_dotenv

load_dotenv(dotenv_path='../../.env')

def load_enviroments():
    load_dotenv()
    braintrust_api_key  =   os.getenv('braintrust_api_key')
    openai_api_key = os.getenv('OPENAI_API_KEY')
    return braintrust_api_key,openai_api_key
braintrust_api_key, openai_api_key = load_enviroments()

os.environ["OPENAI_API_KEY"] = openai_api_key

from braintrust import init_logger, traced, wrap_openai, Eval
from openai import OpenAI

logger = init_logger(project="Podwise", api_key=braintrust_api_key)
client = wrap_openai(OpenAI())  

In [40]:
from typing import List
from pydantic import Field, BaseModel

class QAPair(BaseModel):
    reference: str = Field(..., description="The exact text segment from the original context that this Q&A is based on")
    question: str = Field(description="A single question about the content")
    answer: str = Field(..., description="Answer")

class QAPairs(BaseModel):
    pairs: List[QAPair] = Field(..., description="List of question/answer pairs")

@traced
def produce_questions(content):
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Please generate 2 question/answer pairs from the following text, focusing specifically on podcast recommendations related to business and education topics.
For each pair, provide a single question, a unique answer, and include the exact text segment from the original context that the Q&A is based on.

IMPORTANT:
1. Focus ONLY on business-related, entrepreneurship, management, marketing, leadership, lifelong learning, career development, or educational podcast topics.
2. All questions and answers MUST be in Traditional Chinese (Taiwan).
3. Use terminology and expressions commonly used in Taiwan's business and education sectors.
4. If the context doesn't contain direct podcast recommendations, extract the most relevant aspects that could guide podcast listening decisions for business and education audiences.
5. For each Q&A pair, include the exact text from the original context that contains the information used for the Q&A. This should be copied verbatim from the input context.

Context: <context>{content}</context>""",
            }
        ],
        response_format=QAPairs
    )

    parsed_result = completion.choices[0].message.parsed
    pairs = parsed_result.pairs
    return pairs

In [41]:
x = produce_questions('有哪些頻道有提到黃仁勳嘛?')
x

[QAPair(reference='有哪些頻道有提到黃仁勳嘛?', question='在尋找與業務發展相關的播客推薦時，有哪些頻道會提及黃仁勳？', answer='目前的內容並沒有直接推薦任何涉及黃仁勳的業務或領導相關播客頻道。'),
 QAPair(reference='有哪些頻道有提到黃仁勳嘛?', question='如果要從教育或職涯發展角度挑選播客，這段文字有提供什麼指引嗎？', answer='這段文字未直接提供與教育或職涯發展相關的播客推薦資訊。')]

In [34]:
h = x[0].model_dump()

In [42]:
h['test'] = 1234
h

{'reference': '有哪些頻道有提到黃仁勳嘛?',
 'question': '有哪些頻道會討論與商業與教育相關的話題，比如企業領導與產業趨勢？',
 'answer': '雖然原文僅提到「有哪些頻道有提到黃仁勳嗎？」但由此可以推測，關注科技領袖黃仁勳的頻道往往會涉及商業策略、領導力及產業動態的討論，非常適合想了解企業管理與產業趨勢的聽眾。',
 'test': 1234}

In [52]:
df = pd.read_csv(csv_path, header=1)

dataset = []
for idx, row in df.iterrows():
    context = row['答案']  
    pairs = produce_questions(context)
    for pair in pairs:
        h = pair.model_dump()
        h["row_index"] = idx
        dataset.append(h)


In [46]:
eval_dataset = []
for qa in dataset:
    eval_dataset.append(
        {
            "input": qa['question'],
            "expected": qa['answer'],
            "metadata": {
                "reference": qa.get('reference', ''),  
                'row_index': qa['row_index'],
                'file_name': 'default_QA.csv'
            },
        }
    )


In [47]:
#NoRAG
def simple_qa(question):
    completion = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
    )
    return completion.choices[0].message.content

In [48]:
import autoevals

Eval(
    name="Podwise",
    experiment_name="No RAG",
    data=eval_dataset,
    task=simple_qa,
    scores=[autoevals.Factuality(model="gpt-4.1")],
)

Experiment No RAG-2571c55b is running at https://www.braintrust.dev/app/BDSE/p/Podwise/experiments/No%20RAG-2571c55b
`Eval()` was called from an async context. For better performance, it is recommended to use `await EvalAsync()` instead.


<Task pending name='Task-5' coro=<_EvalCommon.<locals>.run_to_completion() running at /home/bai/Desktop/Podwise/.venv/lib/python3.10/site-packages/braintrust/framework.py:688>>

Podwise [experiment_name=No RAG] (data): 116it [00:00, 63883.83it/s]
Podwise [experiment_name=No RAG] (tasks): 100%|██████████| 116/116 [03:53<00:00,  2.01s/it]



=========================SUMMARY=========================
No RAG-2571c55b compared to No RAG-0a53a921:
56.03% 'Factuality' score

1753066089.96s start
1753066187.87s end
31.69s duration
7.91s llm_duration
28.81tok prompt_tokens
425.50tok completion_tokens
454.31tok total_tokens
0.00$ estimated_cost
0tok prompt_cached_tokens
0tok prompt_cache_creation_tokens

See results for No RAG-2571c55b at https://www.braintrust.dev/app/BDSE/p/Podwise/experiments/No%20RAG-2571c55b


In [53]:
from pymilvus import connections, Collection

# 0. 建立連線（改成你的 host / port / 帳密）
connections.connect(
    alias="default",
    host="192.168.32.86",
    port="19530",
)

# 1. 取得已存在的 collection，並 load 進記憶體
col = Collection("podcast_chunks")   # ← 你的 collection 名
col.load()


In [49]:
!pip install langchain_text_splitters

In [74]:
import tiktoken_ext.openai_public          # 先註冊 OpenAI 編碼
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken


tokenizer = tiktoken.get_encoding("cl100k_base") # gpt-4o 是 o200k_base，之前版本 gpt-4-turbo 和 gpt-3.5-turbo 是 cl100k_base

def length_function(text: str):
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(length_function=length_function, chunk_size=800, chunk_overlap=200, separators=[
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma ，
    "\u3001",  # Ideographic comma 、
    "\uff0e",  # Fullwidth full stop ．
    "\u3002",  # Ideographic full stop 。
    "",
])

In [75]:
def get_embeddings(text):
  response = client.embeddings.create(
      input=text,
      model="bge-m3"
  )

  return response.data[0].embedding

In [78]:
for idx,page in enumerate(df):
  chunks = text_splitter.split_text(page)

  col.add(
    documents = chunks,
    embeddings = [ get_embeddings(chunk) for chunk in chunks ],
    ids=[f"doc-1-page-{idx}-chunk-{x}" for x in range( len(chunks) ) ]
  )

AttributeError: 'Collection' object has no attribute 'add'